# DS 7333 - Quantifying the World - Case Study #1

Real Time Location System
Team: Sean Kennedy, Sterling Beeson, Emil Ramos, Jayson Barker

## Problem Statement and Questions Assigned

Using the OFFLINE data set and two different clustering methods (for C0, CD, and C0 & CD combined) predict the location of the ONLINE data set

Describe how we prepared the data

Describe how we estimated the error and found the best fit (not using the ONLINE data)

Describe the best fit for the data

There are two MACID's located at the same position. Does one give better performance than the other?

What about using them both?

What is the drawback (if any of using this method to real time locate an object)?

Describe a method that may be an improvement based on your percieved drawbacks.

## Data Preparation

Description and narrative goes here

In [ ]:
# Import libraries for data analysis
import numpy as np 
import pandas as pd 
import pandas_profiling as pp
import seaborn as sns
import matplotlib.pyplot as plt
import pathlib as pathlib
import math
import requests
import datetime



In [237]:
# read in the data file
# if IO Pub timeout occurs - run this in terminal: jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
data_file_off = 'offline.final.trace.txt'
data_file_on = 'online.final.trace.txt'

filepath = 'C:/users/Jayson/Documents/'

finalpath = filepath + data_file_off

df_1 = pd.read_table(finalpath, squeeze=True, engine='python', skiprows=2, skipfooter=1)

In [238]:
df_2 = pd.DataFrame([x.split(';') for x in df_1.values])
df_2.tail

<bound method NDFrame.tail of                      0                     1                 2             3   \
0       t=1139643118358  id=00:02:2D:21:0F:33   pos=0.0,0.0,0.0    degree=0.0   
1       t=1139643118744  id=00:02:2D:21:0F:33   pos=0.0,0.0,0.0    degree=0.0   
2       t=1139643119002  id=00:02:2D:21:0F:33   pos=0.0,0.0,0.0    degree=0.0   
3       t=1139643119263  id=00:02:2D:21:0F:33   pos=0.0,0.0,0.0    degree=0.0   
4       t=1139643119538  id=00:02:2D:21:0F:33   pos=0.0,0.0,0.0    degree=0.0   
...                 ...                   ...               ...           ...   
151383  t=1141936869448  id=00:02:2D:21:0F:33  pos=20.0,8.0,0.0  degree=315.7   
151384  t=1141936869700  id=00:02:2D:21:0F:33  pos=20.0,8.0,0.0  degree=315.7   
151385  t=1141936869952  id=00:02:2D:21:0F:33  pos=20.0,8.0,0.0  degree=315.7   
151386  t=1141936870204  id=00:02:2D:21:0F:33  pos=20.0,8.0,0.0  degree=315.7   
151387  t=1141936870456  id=00:02:2D:21:0F:33  pos=20.0,8.0,0.0  degree=315.7  

In [240]:
# Strip out separators and set up the column names
names_1 = [x.split("=")[0] for x in df_2.iloc[1][:4].values]
names_2 = ['MAC_'+str(x) for x in range(len(df_2.iloc[1][4:].values))]

label_names = names_1 + names_2
label_names2 = dict(list(zip(df_2.columns.values, label_names)))

df_2.rename(columns=label_names2, inplace=True)
df_2.head()

t                    id              pos      degree  \
0  t=1139643118358  id=00:02:2D:21:0F:33  pos=0.0,0.0,0.0  degree=0.0   
1  t=1139643118744  id=00:02:2D:21:0F:33  pos=0.0,0.0,0.0  degree=0.0   
2  t=1139643119002  id=00:02:2D:21:0F:33  pos=0.0,0.0,0.0  degree=0.0   
3  t=1139643119263  id=00:02:2D:21:0F:33  pos=0.0,0.0,0.0  degree=0.0   
4  t=1139643119538  id=00:02:2D:21:0F:33  pos=0.0,0.0,0.0  degree=0.0   

                                MAC_0                               MAC_1  \
0  00:14:bf:b1:97:8a=-38,2437000000,3  00:14:bf:b1:97:90=-56,2427000000,3   
1  00:14:bf:b1:97:8a=-38,2437000000,3  00:0f:a3:39:e1:c0=-54,2462000000,3   
2  00:14:bf:b1:97:8a=-38,2437000000,3  00:0f:a3:39:e1:c0=-54,2462000000,3   
3  00:14:bf:b1:97:8a=-38,2437000000,3  00:14:bf:b1:97:90=-52,2427000000,3   
4  00:14:bf:b1:97:8a=-46,2437000000,3  00:0f:a3:39:e1:c0=-55,2462000000,3   

                                MAC_2                               MAC_3  \
0  00:0f:a3:39:e1:c0=-53,2462000000,3  00:14:bf:b1:97:8d=-65,2442000000,3   
1  00:14:bf:b1:97:90=-56,2427000000,3  00:14:bf:3b:c7:c6=-67,2432000000,3   
2  00:14:bf:b1:97:90=-57,2427000000,3  00:14:bf:b1:97:81=-66,2422000000,3   
3  00:0f:a3:39:e1:c0=-54,2462000000,3  00:14:bf:b1:97:81=-64,2422000000,3   
4  00:14:bf:b1:97:90=-57,2427000000,3  00:14:bf:3b:c7:c6=-67,2432000000,3   

                                MAC_4                               MAC_5  \
0  00:14:bf:b1:97:81=-65,2422000000,3  00:14:bf:3b:c7:c6=-66,2432000000,3   
1  00:14:bf:b1:97:81=-66,2422000000,3  00:14:bf:b1:97:8d=-70,2442000000,3   
2  00:14:bf:3b:c7:c6=-69,2432000000,3  00:14:bf:b1:97:8d=-70,2442000000,3   
3  00:14:bf:3b:c7:c6=-68,2432000000,3  00:14:bf:b1:97:8d=-74,2442000000,3   
4  00:0f:a3:39:dd:cd=-66,2412000000,3  00:0f:a3:39:e0:4b=-80,2462000000,3   

   ... MAC_11 MAC_12 MAC_13 MAC_14 MAC_15 MAC_16 MAC_17 MAC_18 MAC_19 MAC_20  
0  ...   None   None   None   None   None   None   None   None   None   None  
1  ...   None   None   None   None   None   None   None   None   None   None  
2  ...   None   None   None   None   None   None   None   None   None   None  
3  ...   None   None   None   None   None   None   None   None   None   None  
4  ...   None   None   None   None   None   None   None   None   None   None  

[5 rows x 25 columns]

In [241]:
# Strip out the preceeding values in each column
df_2['t'] = df_2.t.str.replace("t=", "")
df_2['pos'] = df_2.pos.str.replace("pos=", "")
df_2['degree'] = df_2.degree.str.replace("degree=", "")
df_2['degree'] = pd.to_numeric(df_2.degree, errors='coerce', downcast='float')
df_2['id'] = df_2.id.str.replace("id=", "")

In [242]:
# Enumerate POS since it is a combined value field
d = {}
for i, x in enumerate(df_2.pos.values):
    if x != None:
        temp = x.split(",")
        d[i] = [float(y) for y in temp]
    else:
        d[i] = None

pos = pd.Series(d)
df_2['pos'] = pos

# Enumerate MACID since it is a combined field
l = []
for col in df_2.iloc[:,4:].columns:
    d = {}
    colString = '{}'.format(col)
    for i, x in enumerate(df_2[colString].values):
        if x != None:
            temp = x.split("=")
            macId = temp[0]
            coords = [int(y) for y in temp[1].split(",")]
            d[i] = {macId: [coords[:2], coords[-1]]}
            #import pdb; pdb.set_trace()
        else:
            d[i] = None
    
    df_2[colString] = pd.Series(d)

In [244]:
# Identify nulls across the data set
df_2.count()

t         151388
id        146080
pos       146080
degree    146080
MAC_0     146074
MAC_1     146041
MAC_2     146030
MAC_3     145965
MAC_4     145308
MAC_5     141435
MAC_6     127802
MAC_7      97147
MAC_8      54627
MAC_9      21489
MAC_10      6926
MAC_11      1860
MAC_12       566
MAC_13       218
MAC_14        85
MAC_15        36
MAC_16        13
MAC_17         3
MAC_18         1
MAC_19         1
MAC_20         1
dtype: int64

In [248]:
# Use fillna function specifically pad, to pad the data set by forward filling
df_3 = df_2.fillna(method='pad')

# Confirming all NAs are taken care of - but due to missing data prevelance, we will drop the straggling MACIDs through
df_3.isnull().any()

# Dropping MAC ID columns with a high number of NULLs and lower number of observations
df_3 = df_3.iloc[:,:10]

In [250]:
df_3.tail()

t                 id               pos      degree  \
151383  1141936869448  00:02:2D:21:0F:33  [20.0, 8.0, 0.0]  315.700012   
151384  1141936869700  00:02:2D:21:0F:33  [20.0, 8.0, 0.0]  315.700012   
151385  1141936869952  00:02:2D:21:0F:33  [20.0, 8.0, 0.0]  315.700012   
151386  1141936870204  00:02:2D:21:0F:33  [20.0, 8.0, 0.0]  315.700012   
151387  1141936870456  00:02:2D:21:0F:33  [20.0, 8.0, 0.0]  315.700012   

                                                MAC_0  \
151383  {'00:0f:a3:39:e1:c0': [[-51, 2462000000], 3]}   
151384  {'00:0f:a3:39:e1:c0': [[-53, 2462000000], 3]}   
151385  {'00:0f:a3:39:e1:c0': [[-54, 2462000000], 3]}   
151386  {'00:14:bf:3b:c7:c6': [[-51, 2432000000], 3]}   
151387  {'00:14:bf:3b:c7:c6': [[-56, 2432000000], 3]}   

                                                MAC_1  \
151383  {'00:14:bf:b1:97:8d': [[-56, 2442000000], 3]}   
151384  {'00:14:bf:3b:c7:c6': [[-52, 2432000000], 3]}   
151385  {'00:14:bf:3b:c7:c6': [[-58, 2432000000], 3]}   
151386  {'00:0f:a3:39:e1:c0': [[-53, 2462000000], 3]}   
151387  {'00:0f:a3:39:e1:c0': [[-53, 2462000000], 3]}   

                                                MAC_2  \
151383  {'00:14:bf:b1:97:8a': [[-60, 2437000000], 3]}   
151384  {'00:14:bf:b1:97:8d': [[-62, 2442000000], 3]}   
151385  {'00:14:bf:b1:97:8d': [[-56, 2442000000], 3]}   
151386  {'00:14:bf:b1:97:8a': [[-61, 2437000000], 3]}   
151387  {'00:14:bf:b1:97:8d': [[-61, 2442000000], 3]}   

                                                MAC_3  \
151383  {'00:14:bf:b1:97:90': [[-69, 2427000000], 3]}   
151384  {'00:14:bf:b1:97:81': [[-58, 2422000000], 3]}   
151385  {'00:14:bf:b1:97:8a': [[-60, 2437000000], 3]}   
151386  {'00:14:bf:b1:97:81': [[-60, 2422000000], 3]}   
151387  {'00:14:bf:b1:97:8a': [[-60, 2437000000], 3]}   

                                                MAC_4  \
151383  {'00:0f:a3:39:dd:cd': [[-76, 2412000000], 3]}   
151384  {'00:14:bf:b1:97:8a': [[-61, 2437000000], 3]}   
151385  {'00:14:bf:b1:97:90': [[-69, 2427000000], 3]}   
151386  {'00:14:bf:b1:97:8d': [[-62, 2442000000], 3]}   
151387  {'00:14:bf:b1:97:81': [[-60, 2422000000], 3]}   

                                                MAC_5  
151383  {'02:b7:00:bb:a9:35': [[-67, 2447000000], 1]}  
151384  {'00:0f:a3:39:dd:cd': [[-77, 2412000000], 3]}  
151385  {'00:0f:a3:39:dd:cd': [[-77, 2412000000], 3]}  
151386  {'00:0f:a3:39:dd:cd': [[-77, 2412000000], 3]}  
151387  {'00:14:bf:b1:97:90': [[-68, 2427000000], 3]}